In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()

import pandas as pd
import numpy as np

# Blocking Warning Boxes
import warnings
warnings.filterwarnings("ignore")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/02/15 15:28:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## 1. Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

#### Create a spark data frame 

In [2]:
# make a dictionary of terms
laguage_list = {'language':['Python', 'C++', 'SQL', 'Java', 'JavaScript', 'PHP'],
               'creatures':['unicorn', 'werewolf', 'dragon', 'balrog', 'kobold', 'lich']}
 
# Calling DataFrame constructor on language_list
laguage_df = pd.DataFrame(laguage_list)
laguage_df

,language,creatures
0,Python,unicorn
1,C++,werewolf
2,SQL,dragon
3,Java,balrog
4,JavaScript,kobold
5,PHP,lich


In [3]:
# Making my first Spark Dataframe!
df = spark.createDataFrame(laguage_df)
df

DataFrame[language: string, creatures: string]

#### View the schema of the dataframe

In [4]:
df.printSchema()

root
 |-- language: string (nullable = true)
 |-- creatures: string (nullable = true)



#### Output the shape of the dataframe

In [5]:
print("DataFrame shape: ", df.count(), " x ", len(df.columns))

DataFrame shape:  6  x  2


#### Show the first 5 records in the dataframe

In [6]:
df.select('language', 'creatures').show(5)

+----------+---------+
|  language|creatures|
+----------+---------+
|    Python|  unicorn|
|       C++| werewolf|
|       SQL|   dragon|
|      Java|   balrog|
|JavaScript|   kobold|
+----------+---------+
only showing top 5 rows



## 2. Load the `mpg` dataset as a spark dataframe.
- Create 1 column of output that contains a message like the one below:  
`The 1999 audi a4 has a 4 cylinder engine.`  
For each vehicle.

- Transform the trans column so that it only contains either manual or auto.

In [7]:
# Loading the mpg dataset
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



#### Create 1 column of output that contains a message.

In [13]:
# I will be using spark SQL functions for this, so better import them in!
from pyspark.sql.functions import *

mpg.select( concat(
                    lit("The "),
                    col("year"),
                    lit(" "),
                    col("manufacturer"),
                    lit(" "),
                    col("model"),
                    lit(" has a "),
                    col("cyl"),
                    lit(" cylinder engine."),
                  ).alias("Message")
).show(5, truncate=False)
# the "truncate=False" helps show the whole message;
# otherwise lazy spark cuts off the message!

+-----------------------------------------+
|Message                                  |
+-----------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 2008 audi a4 has a 4 cylinder engine.|
|The 1999 audi a4 has a 6 cylinder engine.|
+-----------------------------------------+
only showing top 5 rows



#### Transform the trans column so that it only contains either manual or auto.

In [15]:
# What does the trans column even look like?
mpg.select('trans').show(5)

+----------+
|     trans|
+----------+
|  auto(l5)|
|manual(m5)|
|manual(m6)|
|  auto(av)|
|  auto(l5)|
+----------+
only showing top 5 rows



In [16]:
# Alright, now to try and answer the problem
# probably a when().otherwise() statement
# looks like string functions work!

mpg.select(
    mpg.trans,
    (
        when(mpg.trans.startswith('auto') , "auto")
        .otherwise("manual")
        .alias("transmission")
    ),
).show(10)

+----------+------------+
|     trans|transmission|
+----------+------------+
|  auto(l5)|        auto|
|manual(m5)|      manual|
|manual(m6)|      manual|
|  auto(av)|        auto|
|  auto(l5)|        auto|
|manual(m5)|      manual|
|  auto(av)|        auto|
|manual(m5)|      manual|
|  auto(l5)|        auto|
|manual(m6)|      manual|
+----------+------------+
only showing top 10 rows



## 3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [17]:
# Loading the tips dataset
tips = spark.createDataFrame(data("tips"))
tips.show(5)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



#### What percentage of observations are smokers?

In [19]:
# Let's try a groupby
tips.groupby("smoker").count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [22]:
# Let's try a rollup so I can get the total?
smokers = tips.rollup("smoker").count()
smokers.show()

+------+-----+
|smoker|count|
+------+-----+
|  null|  244|
|    No|  151|
|   Yes|   93|
+------+-----+



In [28]:
smokers.smoker.show(5)

TypeError: 'Column' object is not callable

In [25]:
smokers.select(smokers.smoker, smokers.count).withColumn("percentage", col('smoker') / col('cty')).show()

+---------------+
|(smoker = null)|
+---------------+
|           null|
|          false|
|          false|
+---------------+



#### Create a column that contains the tip percentage


#### Calculate the average tip percentage for each combination of sex and smoker.

## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [ ]:
from vega_dataset import data
weather = data.seattle_weather()
weather = spark.createDataFrame(data("weather"))
weather.show()